# 📘 Anleitung: Starten des Notebooks

Dieses Notebook wird von GitHub geladen. Um die Analyse zu starten, folgen Sie bitte diesen Schritten:

1. **Umgebung vorbereiten:** Gehen Sie oben im Menü auf **Laufzeit** (Runtime) → **Alle ausführen** (Run all).
2. **Warnung bestätigen:** Wenn das Fenster *"Warnung: Dieses Notebook wurde nicht von Google erstellt"* erscheint, klicken Sie auf **„Trotzdem ausführen“**.
3. **Berechnung starten:** Sobald das Bedienfeld erscheint, klicken Sie unten auf die grüne Schaltfläche **▶ Ausführen**.

---
*Die Warnung ist eine Google-Sicherheitsmaßnahme für GitHub-Dateien. Die grüne Schaltfläche trainiert das KI-Modell und sendet die Prognosen an die App.*

In [ ]:
# @title
# --- Sticky UI Panel (responsive, mobile friendly) ---
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

display(HTML("""
<style>
/* Sticky container */
#lotto_sticky_bar {
  position: sticky;
  top: 0;
  z-index: 9999;
  background: white;
  border: 1px solid #e5e7eb;
  border-radius: 14px;
  padding: 12px;
  margin-bottom: 10px;
  box-shadow: 0 8px 24px rgba(0,0,0,0.06);
}

/* Make notebook width nicer on desktop */
.output-area { max-width: 1100px; }

/* -------- Responsive tweaks -------- */
.lotto-row {
  display: flex;
  gap: 10px;
  flex-wrap: wrap;              /* allow wrap on small screens */
  align-items: center;
}

.lotto-row > .lotto-btn {
  flex: 0 0 auto;               /* default: keep size */
}

.lotto-row > .lotto-progress {
  flex: 1 1 320px;              /* progress takes remaining width */
  min-width: 240px;
}

.lotto-tabs .widget-tab {
  width: 100%;
}

/* Mobile: stack everything, full-width buttons */
@media (max-width: 520px) {
  #lotto_sticky_bar { padding: 10px; border-radius: 12px; }
  .lotto-row { flex-direction: column; align-items: stretch; }
  .lotto-row > .lotto-btn,
  .lotto-row > .lotto-progress {
    width: 100% !important;
    min-width: 0 !important;
  }

  /* Make buttons big & readable */
  .lotto-row button {
    height: 46px !important;
    font-size: 15px !important;
    white-space: nowrap !important;   /* do not wrap text */
  }

  /* Progress label smaller so it fits */
  .lotto-row .widget-label {
    font-size: 12px !important;
  }

  /* Tabs + outputs a bit taller on mobile */
  .lotto-output {
    height: 300px !important;
  }
  .lotto-logs {
    height: 300px !important;
    font-size: 13px !important;
  }

  /* Title smaller */
  .lotto-title-main { font-size: 16px !important; }
  .lotto-title-sub  { font-size: 12px !important; }
}
</style>
"""))

# ----- Widgets -----
title = widgets.HTML(
    value="""
    <div>
      <div class="lotto-title-main" style="font-size:18px;font-weight:800">LOTTO 6aus49 – Runner</div>
      <div class="lotto-title-sub" style="font-size:12.5px;color:#4b5563">
        Klicke auf ▶ Ausführen. Ergebnisse werden mit deinem Account synchronisiert.
      </div>
    </div>
    """
)

btn_run = widgets.Button(
    description="▶ Ausführen",
    button_style="success",
    tooltip="Startet den kompletten Lauf",
    layout=widgets.Layout(width="170px", height="40px")
)
btn_run.add_class("lotto-btn")

btn_clear = widgets.Button(
    description="🧹 Clear",
    tooltip="Leert Ergebnis/Logs",
    layout=widgets.Layout(width="120px", height="40px")
)
btn_clear.add_class("lotto-btn")

progress = widgets.IntProgress(value=0, min=0, max=100, description="Fortschritt:", bar_style="info")
progress.add_class("lotto-progress")

status = widgets.HTML(value="")

out_result = widgets.Output(layout={
    "border":"1px solid #e5e7eb",
    "border_radius":"12px",
    "padding":"10px",
    "height":"260px",
    "overflow":"auto"
})
out_result.add_class("lotto-output")

log_area = widgets.Textarea(
    value="",
    placeholder="Logs erscheinen hier…",
    layout=widgets.Layout(width="100%", height="260px")
)
log_area.add_class("lotto-logs")

tabs = widgets.Tab(children=[out_result, log_area])
tabs.set_title(0, "Ergebnis")
tabs.set_title(1, "Logs")

tabs_box = widgets.VBox([tabs])
tabs_box.add_class("lotto-tabs")

controls = widgets.HBox(
    [btn_run, btn_clear, progress],
    layout=widgets.Layout(gap="10px")
)
controls.add_class("lotto-row")

# Sticky container + UI
display(HTML('<div id="lotto_sticky_bar"></div>'))
display(widgets.VBox([title, controls, status, tabs_box], layout=widgets.Layout(gap="10px")))

# ----- Helper -----
def ui_set_status(html, kind="info"):
    colors = {
        "info":   ("#1f2937", "#eef2ff", "#c7d2fe"),
        "success":("#065f46", "#ecfdf5", "#a7f3d0"),
        "warn":   ("#92400e", "#fffbeb", "#fcd34d"),
        "error":  ("#991b1b", "#fef2f2", "#fecaca"),
    }
    fg, bg, bd = colors.get(kind, colors["info"])
    status.value = f"""
    <div style="margin-top:10px;padding:10px 12px;border-radius:12px;
                border:1px solid {bd}; background:{bg}; color:{fg}; font-size:13px">
      {html}
    </div>
    """

def log(msg):
    log_area.value = (log_area.value + msg + "\n") if log_area.value else (msg + "\n")

def clear_all(_=None):
    progress.value = 0
    status.value = ""
    log_area.value = ""
    with out_result:
        clear_output()

btn_clear.on_click(clear_all)
ui_set_status("Bereit. Klicke auf <b>▶ Ausführen</b>.", "info")

In [ ]:
# Imports
import os
import io
import time
import json
import zipfile
import requests
import urllib.parse as up
from datetime import datetime
import numpy as np
import pandas as pd
import pytz
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

In [ ]:
BACKEND_URL = "https://apilotto.euroceiling39.ru"
MODEL_PARAMS = {"batch_size": 98, "epochs": 123, "window_length": 24}
def log(msg): log_area.value = (log_area.value + msg + "\n") if log_area.value else (msg + "\n")
def clear_all(_=None):
    progress.value = 0
    with out_result: clear_output()
    log_area.value = ""
btn_clear.on_click(clear_all)
# --- Fetch Hash Params (Magic part) ---
def get_url_hash_params():
    try:
        from google.colab import output
        js = """
        (function() {
          var hash = window.location.hash.substring(1);
          var params = {};
          if(hash) {
            hash.split('&').forEach(function(part) {
               var item = part.split('=');
               if (item.length === 2) params[item[0]] = decodeURIComponent(item[1]);
            });
          }
          return params;
        })();
        """
        return output.eval_js(js) or {}
    except: return {}
# --- Download & Model ---
def download_data():
    url = "https://www.lotto-bayern.de/static/gamebroker_2/de/download_files/archiv_lotto.zip"
    r = requests.get(url, timeout=30)
    with zipfile.ZipFile(io.BytesIO(r.content)) as z: z.extractall("/content/tmp")
    rows = []
    with open("/content/tmp/lotto.txt", "r", encoding="utf-8", errors="ignore") as f:
        for line in f.readlines()[1:]:
            p = line.split()
            if len(p)>10 and int(p[2])>=2013: rows.append(p[3:10])
    return pd.DataFrame(rows).apply(pd.to_numeric, errors='coerce').dropna()
def run_pipeline(_):
    # 1. AUTO-FILL ID Check
    if not txt_userid.value:
        params = get_url_hash_params()
        if "userId" in params:
            txt_userid.value = str(params["userId"])

    user_id = txt_userid.value.strip()
    if not user_id:
        with out_result: print("⚠️ Kein User ID gefunden. Bitte Colab über App öffnen.")
        return
    progress.value = 5
    log(f"Starte für User ID: {user_id}")

    try:
        df = download_data()
        vals = df.values; vals[:,:6] = np.sort(vals[:,:6], axis=1)
        scaler = StandardScaler().fit(vals); scaled = scaler.transform(vals)
        win = 24
        X = np.array([scaled[i:i+win] for i in range(len(scaled)-win)])
        y = np.array([scaled[i+win] for i in range(len(scaled)-win)])

        progress.value = 30
        model = Sequential([
            Bidirectional(LSTM(128, return_sequences=True), input_shape=(win, 7)),
            Dropout(0.2), Bidirectional(LSTM(64)), Dense(7)
        ])
        model.compile(optimizer='adam', loss='mse')

        class P(Callback):
            def on_epoch_end(s,e,l): progress.value = 30 + int(60 * (e/123))

        log("Training...")
        model.fit(X, y, epochs=123, batch_size=98, verbose=0, callbacks=[P()])

        last = scaled[-win:].reshape(1, win, 7)
        pred = scaler.inverse_transform(model.predict(last))[0]
        nums = sorted([min(49, max(1, int(round(x)))) for x in pred[:6]])
        super_n = min(9, max(0, int(round(pred[6]))))

        with out_result:
            print(f"✅ User {user_id}: {nums} (S: {super_n})")

        res = requests.post(f"{BACKEND_URL}/api/colab/update", json={
            "userId": user_id, "numbers": nums, "generated_at": int(time.time())
        })
        if res.ok:
            log("✅ Gesendet!"); progress.value = 100
            with out_result: print("🚀 Erfolgreich an App übertragen!")
        else:
            log(f"Err: {res.text}")

    except Exception as e: log(str(e))
btn_run.on_click(run_pipeline)
# Try auto-detect immediately on load
params = get_url_hash_params()
if "userId" in params: txt_userid.value = str(params["userId"])